In [45]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [91]:
#baca data yang dicari sentimentnya

filename = "umk_tweets_202112_clean.csv"
df_tweets = pd.read_csv(f"{filename}")
df_tweets  

,tweet,userid
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658
2,umk yo regone mundak maneh bos,1572322873
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136
4,sumpah umk malang segini iya mending nyari ker...,181023564
...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592
1599,umk solo rp ribu dinilai layak,2392460208
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576


In [92]:
#baca data yang akan jadi training dataset model

filename = "dataset_komentar_instagram_cyberbullying.csv"
df_tweets_model = pd.read_csv(f"{filename}")
df_tweets_model

,Id,Sentiment,Instagram Comment Text
0,1,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,2,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,3,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,4,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,5,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...,...
395,396,positive,Bangga sama suami yg selalu ingat istri disela...
396,397,positive,Apaoun pekerjaannya yg penting halal u tuk men...
397,398,positive,Gojek itu mayoritas pegangguran yang lama gak ...
398,399,positive,<USERNAME> aslinya cantik dan ayu loh mbak kr...


In [93]:
df_tweets_model.rename(columns={"Instagram Comment Text":"Text"}, inplace=True)
df_tweets_model.drop(columns=["Id"],inplace=True,)
df_tweets_model.replace({"Sentiment":{"negative":0, "positive":1}}, inplace=True)

df_tweets

,tweet,userid
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658
2,umk yo regone mundak maneh bos,1572322873
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136
4,sumpah umk malang segini iya mending nyari ker...,181023564
...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592
1599,umk solo rp ribu dinilai layak,2392460208
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576


In [119]:
df_tweets_model["clean_text"] = df_tweets_model["Text"].str.lower()
df_tweets_model

df_tweets["clean_text"] = df_tweets["tweet"].str.lower()
df_tweets

,tweet,userid,clean_text
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,umk sragen ditetapkan rp rp sayang terendah so...
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,umk kota bekasi ump dki jakarta yak terobosss😭
2,umk yo regone mundak maneh bos,1572322873,umk yo regone mundak maneh bos
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,umk kabupatenkota jawa resmi diumumkan kota se...
4,sumpah umk malang segini iya mending nyari ker...,181023564,sumpah umk malang segini iya mending nyari ker...
...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,ngelihat gaji dihitung make idr susah nyentuh ...
1599,umk solo rp ribu dinilai layak,2392460208,umk solo rp ribu dinilai layak
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,bicaranya tidak sesuai kenyataan jakarta warga...
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,dikabarkan persen daftar umk provinsi banten c...


In [95]:
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ").replace("<username>", " ")
                
#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans(" "," ",string.punctuation))

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)


In [109]:
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    #return [word for word in words if word not in list_stopwords]
    
    resultwords = [word for word in words if word not in list_stopwords]
    return ' '.join(resultwords)

In [110]:
df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_tweet_special)
df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_number)
df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_punctuation)
#df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_whitespace_LT)
#df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_whitespace_multiple)
#df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_singl_char)

df_tweets_model["clean_text"] = df_tweets_model["clean_text"].str.split()
df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(stopwords_removal)

df_tweets_model

,Sentiment,Text,clean_text
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,tolol hubungan keguguran pake hijab syari lo b...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......,geblek lo tatacowo bgt dibelain balikanhadewwn...
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn termewek skr lengket lg duhhh labil bgt m...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ...",intinya kesel att gausah anaknya kasian perkem...
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",hadewwwww permpuan lgsakit jiwaknp peran utama...
...,...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...,bangga suami istri disela apapun profesimu kau...
396,1,Apaoun pekerjaannya yg penting halal u tuk men...,apaoun pekerjaannya halal tuk menafkahi keluar...
397,1,Gojek itu mayoritas pegangguran yang lama gak ...,gojek mayoritas pegangguran kerja trus putus a...
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...,aslinya cantik ayu mbak bertemu langsung sesem...


In [120]:
df_tweets["clean_text"] = df_tweets["clean_text"].apply(remove_tweet_special)
df_tweets["clean_text"] = df_tweets["clean_text"].apply(remove_number)
df_tweets["clean_text"] = df_tweets["clean_text"].apply(remove_punctuation)
#df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_whitespace_LT)
#df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_whitespace_multiple)
#df_tweets_model["clean_text"] = df_tweets_model["clean_text"].apply(remove_singl_char)

df_tweets["clean_text"] = df_tweets["clean_text"].str.split()
df_tweets["clean_text"] = df_tweets["clean_text"].apply(stopwords_removal)

df_tweets

,tweet,userid,clean_text
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,umk sragen ditetapkan rp rp sayang terendah so...
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,umk kota bekasi ump dki jakarta yak terobosss
2,umk yo regone mundak maneh bos,1572322873,umk yo regone mundak maneh bos
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,umk kabupatenkota jawa resmi diumumkan kota se...
4,sumpah umk malang segini iya mending nyari ker...,181023564,sumpah umk malang segini iya mending nyari ker...
...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,ngelihat gaji dihitung make idr susah nyentuh ...
1599,umk solo rp ribu dinilai layak,2392460208,umk solo rp ribu dinilai layak
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,bicaranya sesuai kenyataan jakarta warganya me...
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,dikabarkan persen daftar umk provinsi banten c...


In [173]:
vectorizer = TfidfVectorizer(max_features = 2500)
model_g = GaussianNB()

v_data = vectorizer.fit_transform(df_tweets_model["clean_text"]).toarray()
vectorizer.fit_transform(df_tweets_model["clean_text"])

<400x2500 sparse matrix of type '<class 'numpy.float64'>'
	with 7753 stored elements in Compressed Sparse Row format>

In [176]:
X_train, X_test, y_train, y_test = train_test_split(v_data, df_tweets_model['Sentiment'], test_size=0.2, random_state=18)

model_g.fit(X_train,y_train)

y_preds = model_g.predict(X_test)

 

print(confusion_matrix(y_test,y_preds))

print(classification_report(y_test,y_preds))

print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[34  8]
 [ 5 33]]
              precision    recall  f1-score   support

           0       0.87      0.81      0.84        42
           1       0.80      0.87      0.84        38

    accuracy                           0.84        80
   macro avg       0.84      0.84      0.84        80
weighted avg       0.84      0.84      0.84        80

nilai akurasinya adalah  0.8375


In [178]:
v_data = vectorizer.transform(df_tweets["clean_text"]).toarray()
y_classification = model_g.predict(v_data)

df_tweets["sentiment"] = list(y_classification)
df_tweets.to_csv('sentimen_result', index=False)
df_tweets

,tweet,userid,clean_text,sentiment
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,umk sragen ditetapkan rp rp sayang terendah so...,1
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,umk kota bekasi ump dki jakarta yak terobosss,1
2,umk yo regone mundak maneh bos,1572322873,umk yo regone mundak maneh bos,1
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,umk kabupatenkota jawa resmi diumumkan kota se...,1
4,sumpah umk malang segini iya mending nyari ker...,181023564,sumpah umk malang segini iya mending nyari ker...,0
...,...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,ngelihat gaji dihitung make idr susah nyentuh ...,1
1599,umk solo rp ribu dinilai layak,2392460208,umk solo rp ribu dinilai layak,1
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,bicaranya sesuai kenyataan jakarta warganya me...,1
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,dikabarkan persen daftar umk provinsi banten c...,0
